In [1]:
import json
from io import StringIO

import pandas as pd
import numpy as np

# County Data (Note: started using MIT data below instead)

In [2]:
# state_abbrev = {
#     'Alabama': 'AL','Alaska': 'AK','Arizona': 'AZ','Arkansas': 'AR','California': 'CA',
#     'Colorado': 'CO','Connecticut': 'CT','Delaware': 'DE','Florida': 'FL','Georgia': 'GA',
#     'Hawaii': 'HI','Idaho': 'ID','Illinois': 'IL','Indiana': 'IN','Iowa': 'IA','Kansas': 'KS',
#     'Kentucky': 'KY','Louisiana': 'LA','Maine': 'ME','Maryland': 'MD','Massachusetts': 'MA',
#     'Michigan': 'MI','Minnesota': 'MN','Mississippi': 'MS','Missouri': 'MO','Montana': 'MT',
#     'Nebraska': 'NE','Nevada': 'NV','New Hampshire': 'NH','New Jersey': 'NJ','New Mexico': 'NM',
#     'New York': 'NY','North Carolina': 'NC','North Dakota': 'ND','Ohio': 'OH','Oklahoma': 'OK',
#     'Oregon': 'OR','Pennsylvania': 'PA','Rhode Island': 'RI','South Carolina': 'SC',
#     'South Dakota': 'SD','Tennessee': 'TN','Texas': 'TX','Utah': 'UT','Vermont': 'VT',
#     'Virginia': 'VA','Washington': 'WA','West Virginia': 'WV','Wisconsin': 'WI','Wyoming': 'WY',
#     'District of Columbia': 'DC'
# }

In [3]:
# # Load the 2012, 2016 Voting and Turnout data
# # Source: https://github.com/kyaroch/2012_and_2016_presidential_election_results_by_county

# voting1_df = pd.read_csv('./data/2012_and_2016_presidential_election_results_by_county.csv',
#                          converters={'fips_code': lambda x: str(x)})

# columns = ['state_postal_abbrev', 'fips_code', 'county_name', '2012_total_votes', 
#            'obama_votes', 'romney_votes', '2016_total_votes', 'clinton_votes', 
#            'trump_votes', '2012_adult_pop', '2016_extrapolated_adult_pop', 
#            '2012_turnout', '2016_turnout']

# renames = ['state',  'fips_code', 'county_name',  'total_2012', 'dem_2012', 
#            'rep_2012','total_2016', 'dem_2016', 'rep_2016', 'vap_2012','vap_2016',
#            'turnout_2012','turnout_2016']

# voting1_df = voting1_df[columns]
# voting1_df.rename(columns = dict(zip(columns, renames)), inplace=True)

# #Fix fips_code, add leading zero
# voting1_df['fips_code'] = voting1_df['fips_code'].apply(lambda x: '0' + x if len(x) < 5 else x)

# voting1_df['county_name'] = voting1_df['county_name'] + " County"

# # voting1_df[voting1_df['2016_results_official'] == 0].count()
# # 385 unofficial results

# #Close enough to above numbers from Leip's election atlas. 
# print(voting1_df.sum(numeric_only=True).apply(lambda x: '%.2f' % x))

# #Turn year into column
# year_dfs = []

# for year in [2012, 2016]:
#     cols = 'state fips_code county_name total_{0} dem_{0} rep_{0} vap_{0} turnout_{0}'.format(year)
#     cols = cols.split(' ')
#     temp_df = voting1_df[cols].copy()
#     temp_df['year'] = year
#     renames = dict(zip(cols, ['state', 'fips_code', 'county_name',
#                               'county_num', 'dem_num', 'rep_num', 'vap', 'turnout']))
#     temp_df.rename(columns=renames, inplace=True)
#     year_dfs.append(temp_df)

# voting1_df = pd.concat(year_dfs)

# voting1_df['dem_margin'] = (voting1_df['dem_num'] - voting1_df['rep_num'])/voting1_df['county_num']

# #voting1_df.sort_values(by='dem_margin')
# voting1_df.head()


In [4]:
# # Load Voting Age Population 2005 to 2009 average, can't find yearly VAP data
# # Source: https://www.census.gov/rdo/data/voting_age_population_by_citizenship_and_race_cvap.html

# # 'latin-1' deals with e-acute: https://stackoverflow.com/questions/5552555
# population2009_df = pd.read_csv('./data/CVAP_CSV_Format_2005-2009/County.csv', encoding='latin-1')

# population2009_df['fips_code'] = population2009_df['GEOID'].apply(lambda x: x[7:])
# population2009_df = population2009_df[(population2009_df['LNTITLE'] == 'Total')]
# population2009_df.rename(columns={'CVAP_EST':'vap'}, inplace=True)
# population2009_df = population2009_df[['fips_code', 'vap']]  

# population2009_df.head()

In [5]:
# # Load 2004-2012 Voting data 
# # Source: https://github.com/helloworlddata/us-presidential-election-county-results
# columns = ['year', 'state', 'county', 'fips', 'vote_rep', 'vote_dem', 'vote_total']
# renames = ['year', 'state', 'county_name', 'fips_code', 'rep_num', 'dem_num', 'county_num']


# voting2_df = pd.read_csv('./data/us-presidential-election-county-results-2004-through-2012.csv')

# voting2_df = voting2_df[columns]
# voting2_df.rename(columns=dict(zip(columns, renames)), inplace=True)

# # Fix Laclede County Results, otherwise state switches to D
# # https://en.wikipedia.org/wiki/United_States_presidential_election_in_Missouri,_2008
# # Laclede	32.4%	5,218	67.6%	10,875	16,093
# voting2_df.loc[(voting2_df['state'] == 'MO') & 
#            (voting2_df['year'] == 2008) & 
#            (voting2_df['county_name'] == 'Laclede County'), 'county_num'] = 16093

# voting2_df['dem_margin'] = (voting2_df['dem_num'] - voting2_df['rep_num'])/voting2_df['county_num'] 

# voting2_df = voting2_df[(voting2_df['year'].isin([2004, 2008])) &
#                         (voting2_df['state'] != 'AK')]

# voting2_df = pd.merge(voting2_df, population2009_df, on='fips_code')

# voting2_df['turnout'] = voting2_df['county_num'] / voting2_df['vap']

# #Close enough to above numbers from Leip's election atlas. 
# #print(voting2_df[.sum(numeric_only=True).apply(lambda x: '%.2f' % x))


# voting2_df.head()

In [6]:
# # Source: https://en.wikipedia.org/wiki/Electoral_College_(United_States)#Chronological_table
# electoral_df = pd.read_csv('./data/state_electoral_votes.csv') #sep="\t"

# # Electoral points in 2004 = 2008, 2012=2016=2020
# electoral_df['votes_2016'] = electoral_df['votes_2012']
# electoral_df['votes_2004'] = electoral_df['votes_2008']
# electoral_df['state'] = electoral_df['state'].apply(lambda x: state_abbrev[x])

# years = [2004, 2008, 2012, 2016]
# electoral_dfs = []

# #Stack by year
# for year in years:
#     cols = 'state votes_{0}'.format(year)
#     cols = cols.split(' ')
#     temp_df = electoral_df[cols].copy()
#     temp_df['year'] = year
#     temp_df.rename(columns={'votes_{}'.format(year): 'state_electoral_votes'}, inplace=True)
#     electoral_dfs.append(temp_df)

# electoral_df = pd.concat(electoral_dfs)
# electoral_df.head()

In [7]:
# # Combine all dataframes together
# turnout_df = pd.concat([voting1_df, voting2_df, alaska_df])

# #Add in electoral college points for each year
# turnout_df = pd.merge(turnout_df, electoral_df, on=('state', 'year'))

# turnout_df.head()

In [8]:
# #Data output script for interactive visualization, filters out abnormal values  
# out_df = turnout_df[(turnout_df['dem_margin'] <= 1.0) &
#                     (turnout_df['dem_margin'] >= -1.0) &
#                     (turnout_df['turnout'] >= 0.0) &
#                     (turnout_df['turnout'] <= 1.0)]

# # out_df = out_df[['county', 'state', 'county_num', 'turnout', 'num_rep', 'num_dem', 'num_state_dem',
# #                  'num_state_rep','num_state', 'year', 'vap', 'fips_code', 'state_electoral_votes']]

# out_df = out_df.round(decimals=4)

# out_df.to_csv('./US_County_Level_Presidential_Results_04-16.csv', index=False)

# State Electoral Votes

In [2]:
state_abbrev = {
    'Alabama': 'AL','Alaska': 'AK','Arizona': 'AZ','Arkansas': 'AR','California': 'CA',
    'Colorado': 'CO','Connecticut': 'CT','Delaware': 'DE','Florida': 'FL','Georgia': 'GA',
    'Hawaii': 'HI','Idaho': 'ID','Illinois': 'IL','Indiana': 'IN','Iowa': 'IA','Kansas': 'KS',
    'Kentucky': 'KY','Louisiana': 'LA','Maine': 'ME','Maryland': 'MD','Massachusetts': 'MA',
    'Michigan': 'MI','Minnesota': 'MN','Mississippi': 'MS','Missouri': 'MO','Montana': 'MT',
    'Nebraska': 'NE','Nevada': 'NV','New Hampshire': 'NH','New Jersey': 'NJ','New Mexico': 'NM',
    'New York': 'NY','North Carolina': 'NC','North Dakota': 'ND','Ohio': 'OH','Oklahoma': 'OK',
    'Oregon': 'OR','Pennsylvania': 'PA','Rhode Island': 'RI','South Carolina': 'SC',
    'South Dakota': 'SD','Tennessee': 'TN','Texas': 'TX','Utah': 'UT','Vermont': 'VT',
    'Virginia': 'VA','Washington': 'WA','West Virginia': 'WV','Wisconsin': 'WI','Wyoming': 'WY',
    'District of Columbia': 'DC'
}

In [3]:
state_converter = pd.read_csv('./data/state_converter.csv', dtype='object')
state_converter.head()

,state,state_abbr,fips
0,Washington,WA,53
1,Delaware,DE,10
2,District of Columbia,DC,11
3,Wisconsin,WI,55
4,West Virginia,WV,54


In [4]:
# Source: https://en.wikipedia.org/wiki/Electoral_College_(United_States)#Chronological_table
electoral_df = pd.read_csv('./data/state_electoral_votes.csv') #sep="\t"

# Electoral points in 2004 = 2008, 2012=2016=2020
electoral_df['votes_2020'] = electoral_df['votes_2012']
electoral_df['votes_2016'] = electoral_df['votes_2012']
electoral_df['votes_2004'] = electoral_df['votes_2008']
electoral_df['state'] = electoral_df['state'].apply(lambda x: state_abbrev[x])

years = [2004, 2008, 2012, 2016, 2020]
electoral_dfs = []

#Stack by year
for year in years:
    cols = 'state votes_{0}'.format(year)
    cols = cols.split(' ')
    temp_df = electoral_df[cols].copy()
    temp_df['year'] = year
    temp_df.rename(columns={'votes_{}'.format(year): 'state_electoral_votes'}, inplace=True)
    electoral_dfs.append(temp_df)

electoral_df = pd.concat(electoral_dfs)
electoral_df

,state,state_electoral_votes,year
0,AL,9,2004
1,AK,3,2004
2,AZ,10,2004
3,AR,6,2004
4,CA,55,2004
...,...,...,...
46,VA,13,2020
47,WA,12,2020
48,WV,5,2020
49,WI,10,2020


# County Population Data

Citizen voting age population [data](https://www.census.gov/programs-surveys/decennial-census/about/voting-rights/cvap.2020.html) by county, for 2005-2009, 2010-2014, 2014-2018

In [5]:
# Load Voting Age Population 2005 to 2009 average, can't find yearly VAP data
# Source: https://www.census.gov/rdo/data/voting_age_population_by_citizenship_and_race_cvap.html
# 'latin-1' deals with e-acute: https://stackoverflow.com/questions/5552555
population2009_df = pd.read_csv('./data/counties/CVAP_CSV_Format_2005-2009/County.csv', encoding='latin-1')
population2009_df['fips_code'] = population2009_df['GEOID'].apply(lambda x: x[7:])
population2009_df = population2009_df[(population2009_df['LNTITLE'] == 'Total')]
population2009_df.rename(columns={'CVAP_EST':'vap'}, inplace=True)
population2009_df = population2009_df[['fips_code', 'vap']]
population2009_df['year_category'] = '2004-2008'

population2014_df = pd.read_csv('./data/counties/CVAP_2010-2014_ACS_csv_files/County.csv', encoding='latin-1')
population2014_df['fips_code'] = population2014_df['GEOID'].apply(lambda x: x[7:])
population2014_df = population2014_df[(population2014_df['LNTITLE'] == 'Total')]
population2014_df.rename(columns={'CVAP_EST':'vap'}, inplace=True)
population2014_df = population2014_df[['fips_code', 'vap']]
population2014_df['year_category'] = '2012'

population2018_df = pd.read_csv('./data/counties/CVAP_2014-2018_ACS_csv_files/County.csv', encoding='latin-1')
population2018_df['fips_code'] = population2018_df['geoid'].apply(lambda x: x[7:])
population2018_df = population2018_df[(population2018_df['lntitle'] == 'Total')]
population2018_df.rename(columns={'cvap_est':'vap'}, inplace=True)
population2018_df = population2018_df[['fips_code', 'vap']]
population2018_df['year_category'] = '2016-2020'

countypop_df = pd.concat([population2009_df, population2014_df, population2018_df])
countypop_df

,fips_code,vap,year_category
0,01001,35315,2004-2008
13,01003,128945,2004-2008
26,01005,22285,2004-2008
39,01007,16255,2004-2008
52,01009,41095,2004-2008
...,...,...,...
41795,72145,42185,2016-2020
41808,72147,6980,2016-2020
41821,72149,17750,2016-2020
41834,72151,27200,2016-2020


In [6]:
# Only 1 county for DC?
countypop_df[countypop_df['fips_code'].str.contains('^11')]

,fips_code,vap,year_category
4147,11001,430110,2004-2008
4147,11001,475400,2012
4147,11001,511190,2016-2020


# MIT County Margin Data

MIT Election labs have since [started](https://dataverse.harvard.edu/dataset.xhtml?persistentId=doi:10.7910/DVN/VOQCHQ) compiling this data, so use that instead. 


* TODO: Wait for more official county data? Because this looks weird, very scattered for turnout
* TODO: Interpolate/project CVAP, so turnout is more accurate?

In [37]:
county = pd.read_csv(
    './data/counties/mit_countydata/countypres_2000-2016.csv',
    dtype={'year':'int', 'FIPS':'object'}
)
county['fips_code'] = county['FIPS'].str.zfill(5)

county = county[
    (county['year'] > 2000) &
    (county['party'].isin(['democrat', 'republican']))
]

county = county.pivot_table(
    index=['year', 'state', 'state_po', 'fips_code', 'county', 'totalvotes'],
    columns='party',
    values='candidatevotes'
)

county.reset_index(drop=False, inplace=True)
del county['state']
county.rename(columns={'state_po':'state',
    'county':'county_name', 'democrat':'dem_num', 
    'republican':'rep_num', 'totalvotes':'county_num'},
    inplace=True)
cols = ['year', 'state', 'county_name', 'fips_code', 'rep_num', 'dem_num', 'county_num']
county = county[cols]
county

party,year,state,county_name,fips_code,rep_num,dem_num,county_num
0,2004,AL,Autauga,01001,15196.0,4758.0,20081
1,2004,AL,Baldwin,01003,52971.0,15599.0,69320
2,2004,AL,Barbour,01005,5899.0,4832.0,10777
3,2004,AL,Bibb,01007,5472.0,2089.0,7600
4,2004,AL,Blount,01009,17386.0,3938.0,21504
...,...,...,...,...,...,...,...
12610,2016,WY,Sweetwater,56037,12154.0,3231.0,17130
12611,2016,WY,Teton,56039,3921.0,7314.0,12627
12612,2016,WY,Uinta,56041,6154.0,1202.0,8470
12613,2016,WY,Washakie,56043,2911.0,532.0,3814


In [38]:
# Get 2020 data by county, also courtesy of MIT:
# Doesn't seem like this source is accurate
# https://github.com/MEDSL/election_night2020/blob/main/results/leip_natl_data.csv
# county_2020 = pd.read_csv(
#     './data/counties/mit_countydata/leip_natl_data_2020.csv',
#     dtype={'FIPS':'object'}
# )
# county_2020['fips_code'] = county_2020['FIPS'].str.zfill(5)
# county_2020['state_fips'] = county_2020['fips_code'].apply(lambda x: x[0:2])
# county_2020 = county_2020.merge(
#     state_converter[['fips', 'state_abbr']],
#     left_on='state_fips', right_on='fips'
# )
# county_2020.rename(columns={'state_abbr':'state',
#     'Geographic Name':'county_name', 'Donald J. Trump':'rep_num', 
#     'Joseph R. Biden Jr.':'dem_num', 'Total Vote':'county_num'},
#     inplace=True)
# county_2020['year'] = 2020
# cols = ['year', 'state', 'county_name', 'fips_code', 'rep_num', 'dem_num', 'county_num']
# county_2020 = county_2020[cols]
# county_2020

# Either there was a huge turnout, VAP data is too old from 2018, 
# or MIT data is really off. Trying this other source:
# https://github.com/tonmcg/US_County_Level_Election_Results_08-20/blob/master/2020_US_County_Level_Presidential_Results.csv
# county_2020 = pd.read_csv(
#     './data/counties/2020_US_County_Level_Presidential_Results.csv',
#     dtype={'county_fips':'object'}
# )
# county_2020['state_fips'] = county_2020['county_fips'].apply(lambda x: x[0:2])
# county_2020 = county_2020.merge(
#     state_converter[['fips', 'state_abbr']],
#     left_on='state_fips', right_on='fips'
# )
# county_2020.rename(columns={'state_abbr':'state',
#     'votes_dem':'dem_num', 'votes_gop':'rep_num',
#     'total_votes':'county_num', 'county_fips':'fips_code'},
#     inplace=True)
# county_2020['year'] = 2020
# cols = ['year', 'state', 'county_name', 'fips_code', 'rep_num', 'dem_num', 'county_num']
# county_2020 = county_2020[cols]
# county_2020


# Trying NYTimes data, to allow projecting forward to 100% reporting.
# This does change things, but the end result is that the High Density areas become
# substantially more conservative, which makes sense if the outstanding vote is still
# mostly Democratic. So don't do this for now, just go with existing county data.
#https://static01.nyt.com/elections-assets/2020/data/api/2020-11-03/national-map-page/national/president.json
with open('./data/counties/20201016-nytimes-president.json', 'r') as f:
    county_obj = json.loads(f.read())

county_dfs = []
for race in county_obj['data']['races']:
    temp = pd.DataFrame(race['counties'])
    temp['state'] = race['state_id']
    county_dfs.append(temp)

# Use reported numbers
county_2020 = pd.concat(county_dfs)
county_2020['dem_reported'] = county_2020['results'].apply(
    lambda x: x.get('bidenj')
)
county_2020['rep_reported'] = county_2020['results'].apply(
    lambda x: x.get('trumpd')
)

# Create simple projections, assuming the same fraction as
# reported to each candidate.
# county_2020['dem_num'] = (county_2020['dem_reported']/county_2020['votes'])*county_2020['tot_exp_vote']
# county_2020['rep_num'] = (county_2020['rep_reported']/county_2020['votes'])*county_2020['tot_exp_vote']
# Instead of projecting, just use current values
county_2020['dem_num'] = county_2020['dem_reported']
county_2020['rep_num'] = county_2020['rep_reported']
county_2020['year'] = 2020
cols = ['year', 'state', 'name', 'fips', 'rep_reported', 'dem_reported', 'votes'] #Fixed bug, votes, not tot_exp_votes 
renames = ['year', 'state', 'county_name', 'fips_code', 'rep_num', 'dem_num', 'county_num']
county_2020 = county_2020[cols]
county_2020.rename(columns=dict(zip(cols, renames)), inplace=True)
# Remove AK
county_2020 = county_2020[county_2020['state'] != 'AK']

# Replace DC with single entity, to match census data
dc_2020 = county_2020[county_2020['state'] == 'DC']
dc_2020 = dc_2020.groupby(by=['year', 'state'])[['rep_num', 'dem_num', 'county_num']].sum()
dc_2020.reset_index(drop=False, inplace=True)
dc_2020[['county_name', 'fips_code']] = ['District of Columbia', '11001']

county_2020 = county_2020[county_2020['state'] != 'DC']
county_2020 = pd.concat([county_2020, dc_2020])
county_2020

,year,state,county_name,fips_code,rep_num,dem_num,county_num
0,2020,AL,Autauga,01001,19764,7450,27639
1,2020,AL,Baldwin,01003,83055,24344,108945
2,2020,AL,Barbour,01005,5605,4772,10457
3,2020,AL,Bibb,01007,7508,1982,9573
4,2020,AL,Blount,01009,24595,2627,27459
...,...,...,...,...,...,...,...
19,2020,WY,Teton,56039,4341,9848,14677
20,2020,WY,Uinta,56041,7496,1591,9402
21,2020,WY,Washakie,56043,3245,651,4012
22,2020,WY,Weston,56045,3107,360,3542


In [39]:
county_2020[['county_num', 'dem_num', 'rep_num']].sum()

county_num    154395222
dem_num        78639569
rep_num        72982751
dtype: int64

In [40]:
# Only AK removed
set(electoral_df['state']) - set(county_2020['state'])

{'AK'}

In [41]:
# Manually input Alaska statewide data because they don't use counties
# http://uselectionatlas.org/RESULTS/compare.php?year=2016&fips=2&f=0&off=0&elect=0&type=state
# https://www.census.gov/library/visualizations/2016/comm/citizen_voting_age_population/cb16-tps18_alaska.html
columns = ['fips_code', 'county_num', 'dem_num', 'rep_num', 
    'year', 'state', 'county_name', 'vap']
ak = [
    ['02', '02', '02', '02', '02'],
    [312598, 326197, 300495, 318608, 349411],
    [111025, 123594, 122640, 116454, 150155], 
    [190889, 193841, 164676, 163387, 185807],
    [2004, 2008, 2012, 2016, 2020],
    ['AK', 'AK', 'AK', 'AK', 'AK'],
    ['Alaska', 'Alaska', 'Alaska', 'Alaska', 'Alaska'],
    [550189, 550189, 550189, 550189, 528248]
]
alaska_df = pd.DataFrame(dict(zip(columns, ak)))
alaska_df

,fips_code,county_num,dem_num,rep_num,year,state,county_name,vap
0,02,312598,111025,190889,2004,AK,Alaska,550189
1,02,326197,123594,193841,2008,AK,Alaska,550189
2,02,300495,122640,164676,2012,AK,Alaska,550189
3,02,318608,116454,163387,2016,AK,Alaska,550189
4,02,349411,150155,185807,2020,AK,Alaska,528248


In [42]:
# Combine 2020 data with 2004-2016 data, get VAP data
def get_yearcategory(el):
    if el in [2004, 2008]:
        return '2004-2008'
    elif el in [2012]:
        return '2012'
    elif el in [2016, 2020]:
        return '2016-2020'
    else:
        return None
    
county = pd.concat([county, county_2020])
county['year_category'] = county['year'].apply(get_yearcategory)
# # premerge = county.copy()
# # https://stackoverflow.com/questions/36891977/diff-of-two-dataframes
# # missing = premerge.merge(county, on=['year', 'fips_code'], how='outer', indicator=True)
# # missing[missing['_merge'] == 'left_only']['state_x'].value_counts()
# # Ok, it's mostly Alaska missing. Just append it separately.
# # AK    188
# # MO      4
# # SD      1
# # VA      1
# Merge on county VAP data
county = county.merge(countypop_df, on=['fips_code', 'year_category'])
county = pd.concat([county, alaska_df])
county['dem_margin'] =  (county['dem_num'] - county['rep_num'])/county['county_num']
county['turnout'] = county['county_num']/ county['vap']
county = county[['year', 'state', 'county_name', 
    'fips_code', 'rep_num', 'dem_num', 'county_num', 'vap',
    'dem_margin', 'turnout']]
county = county.merge(electoral_df, on=['state', 'year'])
county.reset_index(drop=True, inplace=True)
county

,year,state,county_name,fips_code,rep_num,dem_num,county_num,vap,dem_margin,turnout,state_electoral_votes
0,2004,AL,Autauga,01001,15196.0,4758.0,20081,35315,-0.519795,0.568625,9
1,2004,AL,Baldwin,01003,52971.0,15599.0,69320,128945,-0.539123,0.537594,9
2,2004,AL,Barbour,01005,5899.0,4832.0,10777,22285,-0.099007,0.483599,9
3,2004,AL,Bibb,01007,5472.0,2089.0,7600,16255,-0.445132,0.467548,9
4,2004,AL,Blount,01009,17386.0,3938.0,21504,41095,-0.625372,0.523275,9
...,...,...,...,...,...,...,...,...,...,...,...
15573,2020,WY,Teton,56039,4341.0,9848.0,14677,16730,0.375213,0.877286,3
15574,2020,WY,Uinta,56041,7496.0,1591.0,9402,14025,-0.628058,0.670374,3
15575,2020,WY,Washakie,56043,3245.0,651.0,4012,6135,-0.646560,0.653953,3
15576,2020,WY,Weston,56045,3107.0,360.0,3542,5460,-0.775551,0.648718,3


In [43]:
county[county['county_name'] == 'Dane']
#So this is 394780, vs estimated VAP of 345,604/0.8 = 432005 based on this article:, 87 vs 80pct turnout diff.
#https://madison.com/wsj/news/local/govt-and-politics/turnout-sets-records-with-dane-county-at-80-madison-at-75/article_e7d5acba-d6e6-5188-9cc0-a540e8afa64f.html

,year,state,county_name,fips_code,rep_num,dem_num,county_num,vap,dem_margin,turnout,state_electoral_votes
6056,2004,WI,Dane,55025,90369.0,181052.0,274249,357240,0.330659,0.767688,10
6128,2008,WI,Dane,55025,73065.0,205984.0,282939,357240,0.469780,0.792014,10
9267,2012,WI,Dane,55025,83644.0,216071.0,304181,372315,0.435356,0.816999,10
15399,2016,WI,Dane,55025,71279.0,217526.0,310017,394780,0.471739,0.785291,10
15471,2020,WI,Dane,55025,78800.0,260185.0,344791,394780,0.526072,0.873375,10


In [18]:
# Checking the county count to find missing 2020 counties
# res1 = county_2020.groupby(by='state')['fips_code'].count().to_frame()
# res2 = county[county['year'] == 2020].groupby('state')['fips_code'].count().to_frame()
# res = res1.merge(res2, on='state')
# res['diff'] = res['fips_code_x'] - res['fips_code_y']
# res

In [45]:
#Data output script for interactive visualization, filters out abnormal values  
out_df = county[
    (county['dem_margin'] <= 1.0) &
    (county['dem_margin'] >= -1.0) &
    (county['turnout'] >= 0.0) &
    (county['turnout'] <= 1.0)
]
# # Clip instead?
# out_df = county.copy()
# out_df['dem_margin'] = out_df['dem_margin'].clip(lower=-1.0, upper=1.0)
# out_df['turnout'] = out_df['turnout'].clip(lower=0.0, upper=1.0)
out_df = out_df.round(decimals=4)
out_df.to_csv('./US_County_Level_Presidential_Results_04-20.csv', index=False)
out_df.describe()

,year,rep_num,dem_num,county_num,vap,dem_margin,turnout,state_electoral_votes
count,15544.000000,1.554400e+04,1.554400e+04,1.554400e+04,1.554400e+04,15544.000000,15544.000000,15544.000000
mean,2011.994339,2.047738e+04,2.175157e+04,4.328340e+04,7.044548e+04,-0.242571,0.607465,13.736104
std,5.655213,4.695298e+04,7.899673e+04,1.247609e+05,2.048493e+05,0.298256,0.100151,10.798975
min,2004.000000,5.400000e+01,5.000000e+00,6.400000e+01,8.000000e+01,-0.930900,0.026000,3.000000
25%,2008.000000,3.098750e+03,1.495750e+03,5.015750e+03,8.440000e+03,-0.467400,0.538575,6.000000
50%,2012.000000,6.908500e+03,3.903000e+03,1.115900e+04,1.949000e+04,-0.267850,0.606300,10.000000
75%,2016.000000,1.749650e+04,1.126925e+04,2.960200e+04,5.073625e+04,-0.054100,0.673425,16.000000
max,2020.000000,1.121538e+06,2.980478e+06,4.188764e+06,6.272375e+06,0.874100,1.000000,55.000000


# State and National

In [46]:
state_df = county[
    (county['dem_margin'] <= 1.0) &
    (county['dem_margin'] >= -1.0) &
    (county['turnout'] >= 0.0) &
    (county['turnout'] <= 1.0)
]
# # Clip instead?
# state_df = county.copy()
# state_df['dem_margin'] = state_df['dem_margin'].clip(lower=-1.0, upper=1.0)
# state_df['turnout'] = state_df['turnout'].clip(lower=0.0, upper=1.0)
state_df = state_df.round(decimals=4)
state_df = state_df.groupby(['state','year']).agg(
    {'county_num':'sum', 'dem_num':'sum','rep_num':'sum',
     'vap':'sum', 'state_electoral_votes':'max'})

state_df = state_df.round(decimals=4)
state_df = state_df.reset_index()
state_df.rename(columns={'county_num':'state_num'}, inplace=True)
state_df

,state,year,state_num,dem_num,rep_num,vap,state_electoral_votes
0,AK,2004,324078,115490.0,197521.0,747494,3
1,AK,2008,331335,126068.0,196377.0,747494,3
2,AK,2012,308459,125721.0,169220.0,761679,3
3,AK,2016,325748,120605.0,165574.0,762934,3
4,AK,2020,349411,150155.0,185807.0,528248,3
...,...,...,...,...,...,...,...
250,WY,2004,243428,70776.0,167629.0,389900,3
251,WY,2008,256035,82868.0,164958.0,389900,3
252,WY,2012,249061,69286.0,170962.0,427305,3
253,WY,2016,255849,55973.0,174419.0,432285,3


In [47]:
state_df.to_csv('./US_State_Level_Presidential_Results_04-20.csv', index=False)

In [48]:
# This filter has weird effects, not sure why
# national_df = county[
#     (county['dem_margin'] <= 1.0) &
#     (county['dem_margin'] >= -1.0) &
#     (county['turnout'] >= 0.0) &
#     (county['turnout'] <= 1.0)
# ].copy()
# OK, so the problem is that some places have turnout that is too high
national_df = county.copy()
national_df = national_df.groupby(['year']).agg(
    {'county_num':'sum', 'dem_num':'sum','rep_num':'sum',
     'vap':'sum'})
national_df.rename(columns={'county_num':'num_nation'}, inplace=True)
national_df = national_df.reset_index()
national_df
# 2020, 2.5 million are missing. Alaska? Other yet uncalled states?
# So with new data, things aren't missing anymore.

,year,num_nation,dem_num,rep_num,vap
0,2004,122046363,58739173.0,61822463.0,208467944
1,2008,131061479,69336086.0,59915583.0,208467944
2,2012,128901516,65639895.0,60634256.0,220303339
3,2016,136405085,65749471.0,62962853.0,229147199
4,2020,154744633,78789724.0,73168558.0,228915863


# Demographic Data

# ANES 2016 Race Data

In [13]:
## Calculate 2016 Democratic Margin By Race
## source: ANES 2016 time series study:
## http://www.electionstudies.org/studypages/anes_timeseries_2016/anes_timeseries_2016.htm

anes_df = pd.read_csv('./data/anes/anes_timeseries_2016/anes_timeseries_2016_rawdata.txt',
                      sep='|')

columns = ['V160102', 'V161310x', 'V162058x']
labels = ['weight_postfull', 'group', 'party']

anes_df = anes_df[columns]
anes_df.rename(columns=dict(zip(columns, labels)), inplace=True)

anes_df = anes_df[anes_df['party'].isin([10,11,12])]  #Filter out nonvoters

group_df = anes_df.groupby(['group', 'party']).agg({'weight_postfull':'sum'})
group_df = group_df.unstack(level=-1)
group_df.columns = group_df.columns.droplevel(0)
group_df = group_df.rename_axis(None, axis=1)

group_df.loc[6] = group_df.loc[[3,4,6,-9]].sum()  #Include nonresponders in other, -9
rows = [1,2,5,6]
races = ['Non-Hispanic White', 'Non-Hispanic Black', 'Hispanic', 'Other']
group_df.rename(index=dict(zip(rows,races)), inplace=True)
group_df = group_df.loc[races] 

group_df.reset_index(inplace=True)
group_df['total'] = group_df[[10,11,12]].sum(axis=1)
group_df.rename(columns={10:'d',11:'r',12:'o'}, inplace=True)

group_df['dem_frac'] = group_df['d']/group_df['total']
group_df['rep_frac'] = group_df['r']/group_df['total']
group_df['other_frac'] = group_df['o']/group_df['total']
#group_df['dem_margin'] = group_df['dem_frac'] - group_df['rep_frac']

group_df['year'] = 2016

exp_df = group_df.copy()
group_df = group_df[['group','year','dem_frac', 'rep_frac']]
group_df


/Users/psthomas/miniconda2/envs/datascience3/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (790,1129,1131) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,group,year,dem_frac,rep_frac
0,Non-Hispanic White,2016,0.393836,0.538035
1,Non-Hispanic Black,2016,0.899861,0.063772
2,Hispanic,2016,0.686294,0.238819
3,Other,2016,0.573222,0.314705


In [169]:
exp_df = exp_df.merge(national_df, on='year')
exp_df['electorate_frac'] = exp_df['total'] / exp_df['total'].sum()

exp_df['altdemnum'] = exp_df['electorate_frac']*exp_df['dem_frac']*exp_df['num_nation']
exp_df['altrepnum'] = exp_df['electorate_frac']*exp_df['rep_frac']*exp_df['num_nation']

print(exp_df['altdemnum'].sum()/exp_df['num_nation'].mean())
print(exp_df['altrepnum'].sum()/exp_df['num_nation'].mean())

print(exp_df['dem_num'].mean()/exp_df['num_nation'].mean())
print(exp_df['rep_num'].mean()/exp_df['num_nation'].mean())
# exp_df

0.49001285898
0.44153235198
0.481491655712
0.460532572669


The results of the demographics extrapolated don't agree with the 
actual cumulative results:  
Extrapolated:  
D: 0.49001285898  
R: 0.44153235198  
Actual:  
D: 0.481491655712  
R: 0.460532572669  

This is a commonly known problem: 

https://www.americanprogress.org/issues/democracy/reports/2017/11/01/441926/voter-trends-in-2016/

> The most vexing issue for electoral analysts exploring these voting trends is determining which, if any, of the existing data sources provide the best and most reliable information on vote composition, turnout, and support rates. For this project, we developed original turnout and support estimates by combining a multitude of publicly available data sources including the American Communities Survey (ACS), the November supplement of the Current Population Survey (CPS), the American National Election Study (ANES), the Cooperative Congressional Election Survey (CCES), our own post-election polling, and voter files from several states.
We used this approach to help address what we believe are systematic problems with some of the most widely available and most frequently cited pieces of data about elections—mainly, that some of the most reliable sources of data we have on demographics do not fit well together with the best data we have on turnout rates, leading to results that vary from the actual levels of turnout seen on Election Day. Furthermore, if we combine those data with the best data we have on vote choice, we get election results that do not line up with reality. This is not due to any one source of information being particularly biased; rather, each particular source has points of weakness. To overcome this, we created a new method for combining these data in ways that fit with known outcomes.

## ANES and CPS 2004-2012 Race Data

In [14]:
#Sources:
#http://www.electproject.org/home/voter-turnout/demographics
#https://docs.google.com/spreadsheets/d/1l5fpK7ysQhQbZPv9hnZ_-PO1J1zBVPXSSQjNejTXecY/edit#gid=0
#Below includes both corrected and uncorrected data, as data corrected for turnout response
#bias doesn't result in correct overall D/R voter percentages.
#Neither corrected or uncorrected result in right count.  
#These turnouts are for citizen voting age population:

# Corrected USEP turnout data:
# race_turnout = StringIO("""
# Turnout Rate	2016	2014	2012	2010	2008	2006	2004
# Non-Hispanic White	64.7%	40.8%	61.8%	45.0%	65.2%	44.7%	64.3%
# Non-Hispanic Black	59.9%	36.4%	67.4%	41.6%	69.1%	36.6%	61.4%
# Hispanic	44.9%	21.1%	43.1%	26.6%	46.5%	25.5%	42.9%
# Other	46.3%	24.5%	45.4%	30.7%	48.0%	28.5%	44.9%
# """)

#Uncorrected USEP turnout data:
race_turnout = StringIO("""
Turnout Rate	2016	2014	2012	2010	2008	2006	2004
Non-Hispanic White	65.3%	45.8%	64.1%	48.6%	66.1%	51.6%	67.2%
Non-Hispanic Black	59.3%	40.4%	66.4%	43.8%	65.2%	41.2%	60.3%
Hispanic	47.6%	27.0%	48.0%	31.2%	49.9%	32.3%	47.2%
Other	49.0%	29.5%	49.0%	34.1%	49.5%	33.4%	48.0%
""")

# Corrected USEP share of electorate data:
# race_share = StringIO("""
# Share of Electorate	2016	2014	2012	2010	2008	2006	2004
# Non-Hispanic White	73.6%	76.9%	74.1%	77.9%	76.6%	81.0%	79.5%
# Non-Hispanic Black	12.3%	11.9%	13.2%	11.4%	12.3%	10.0%	11.1%
# Hispanic	9.1%	7.0%	8.3%	6.8%	7.3%	5.6%	5.9%
# Other	5.0%	4.2%	4.4%	3.9%	3.8%	3.4%	3.6%
# """)

#Try uncorrected USEP data:
race_share = StringIO("""
Share of Electorate	2016	2014	2012	2010	2008	2006	2004
Non-Hispanic White	73.3%	76.3%	73.7%	77.5%	76.3%	80.4%	79.2%
Non-Hispanic Black	12.4%	12.1%	13.4%	11.7%	12.4%	10.3%	11.2%
Hispanic	9.2%	7.3%	8.4%	6.9%	7.4%	5.8%	6.0%
Other	5.0%	4.3%	4.5%	3.9%	3.9%	3.4%	3.6%
""")

#Source:
#http://www.electionstudies.org/nesguide/text/t9a_1_1.txt
race_demmargin = StringIO("""
percent_of_group,1948,1952,1954,1956,1958,1960,1962,1964,1966,1968,1970,1972,1974,1976,1978,1980,1982,1984,1986,1988,1990,1992,1994,1996,1998,2000,2002,2004,2008,2012
Non-Hispanic White,53,40,,39,,48,,65,,41,,30,,46,,36,,36,,39,,52,,51,,46,,42,44,43,
Non-Hispanic Black,65*,80,,64*,,71,,100,,97,,87,,95,,93,,91,,92,,95,,99,,92,,89,100,97,
Hispanic,,,,,,,,,,40*,,50*,,86*,,58*,,52,,65,,65,,83,,59,,61,73,73,
Other,,,,,,,,,,33*,,70*,,43*,,25*,,50*,,83*,,44*,,54*,,57,,36*,82,61
""")

race_repmargin = StringIO("""
percent_of_group,2004,2008,2012
Non-Hispanic White,58,56,57 
Non-Hispanic Black,11,0,3 
Hispanic,39,27,27 
Other,64,18,39 
""")

raceturnout_df = pd.read_csv(race_turnout, sep='\t') #.T
raceturnout_df = pd.melt(raceturnout_df, id_vars=["Turnout Rate"], 
                  var_name="year", value_name="turnout")
raceturnout_df.rename(columns={'Turnout Rate':'group'}, inplace=True)

raceshare_df = pd.read_csv(race_share, sep='\t')
raceshare_df = pd.melt(raceshare_df, id_vars=["Share of Electorate"], 
                  var_name="year", value_name="electorate_frac")
raceshare_df.rename(columns={'Share of Electorate':'group'}, inplace=True)

racedemmargin_df = pd.read_csv(race_demmargin, sep=',',index_col=False)
racedemmargin_df = racedemmargin_df[['percent_of_group', '2004', '2008', '2012']]
racedemmargin_df = pd.melt(racedemmargin_df, id_vars=["percent_of_group"], 
                  var_name="year", value_name="dem_frac")
racedemmargin_df.rename(columns={'percent_of_group':'group'}, inplace=True)

racerepmargin_df = pd.read_csv(race_repmargin, sep=',',index_col=False)
racerepmargin_df = racerepmargin_df[['percent_of_group', '2004', '2008', '2012']]
racerepmargin_df = pd.melt(racerepmargin_df, id_vars=["percent_of_group"], 
                  var_name="year", value_name="rep_frac")
racerepmargin_df.rename(columns={'percent_of_group':'group'}, inplace=True)

race_df = raceturnout_df.merge(raceshare_df, on=['group','year'], how='inner')
race_df = race_df.merge(racedemmargin_df, on=['group','year'], how='inner')
race_df = race_df.merge(racerepmargin_df, on=['group','year'], how='inner')
race_df = race_df.replace({'\%': '', '\*':''}, regex=True)
race_df = race_df.apply(pd.to_numeric, errors='ignore', axis=0)
race_df[['turnout', 'electorate_frac',
         'dem_frac', 'rep_frac']] = race_df[['turnout', 'electorate_frac', 
                                             'dem_frac','rep_frac']] / 100

#Add in 2016 data from ANES
int_df = raceturnout_df.merge(raceshare_df, on=['group','year'], how='inner')
int_df = int_df.replace({'\%': '', '\*':''}, regex=True)
int_df = int_df.apply(pd.to_numeric, errors='ignore', axis=0)
int_df = int_df.merge(group_df, on=['group','year'], how='inner')
int_df['electorate_frac'] = int_df['electorate_frac']/100
int_df['turnout'] = int_df['turnout']/100
race_df = pd.concat([race_df, int_df])

electorate_df = national_df[['year','num_nation']]
race_df = race_df.merge(electorate_df, on='year', how='left')

race_df['dem_margin'] = race_df['dem_frac'] - race_df['rep_frac']
#race_df['num_group'] = race_df['electorate_frac']*race_df['num_nation']
#race_df['num_dem'] = race_df['dem_frac']*race_df['num_group']  #Calculate when loaded into JS
#race_df['num_rep'] = race_df['rep_frac']*race_df['num_group']  #Calculate when loaded into JS
#e.g. race_df['num_dem'] = race_df['dem_frac']*race_df['num_nation']*race_df['electorate_frac']

race_df['demographic'] = 'race'
order = ['demographic','group', 'year','dem_frac','rep_frac','electorate_frac', 'num_nation', 'dem_margin', 'turnout']
race_df = race_df[order]

race_df

,demographic,group,year,dem_frac,rep_frac,electorate_frac,num_nation,dem_margin,turnout
0,race,Non-Hispanic White,2012,0.430000,0.570000,0.737,129254383,-0.140000,0.641
1,race,Non-Hispanic Black,2012,0.970000,0.030000,0.134,129254383,0.940000,0.664
2,race,Hispanic,2012,0.730000,0.270000,0.084,129254383,0.460000,0.480
3,race,Other,2012,0.610000,0.390000,0.045,129254383,0.220000,0.490
4,race,Non-Hispanic White,2008,0.440000,0.560000,0.763,131049863,-0.120000,0.661
5,race,Non-Hispanic Black,2008,1.000000,0.000000,0.124,131049863,1.000000,0.652
6,race,Hispanic,2008,0.730000,0.270000,0.074,131049863,0.460000,0.499
7,race,Other,2008,0.820000,0.180000,0.039,131049863,0.640000,0.495
8,race,Non-Hispanic White,2004,0.420000,0.580000,0.792,122217180,-0.160000,0.672
9,race,Non-Hispanic Black,2004,0.890000,0.110000,0.112,122217180,0.780000,0.603


In [15]:
#out_df = out_df.round(decimals=4), don't round, numbers are huge
#race_df.to_csv('./US_Race_Presidential_Results_04-16.csv', index=False)

# ANES and CPS 2004-2012, 2016 Gender Data

In [16]:
## Calculate 2016 Democratic Margin By Gender
## source: ANES time series cumulative:
## http://www.electionstudies.org/studypages/anes_timeseries_cdf/anes_timeseries_cdf.htm
anes2016_df = pd.read_csv('./data/anes/anes_timeseries_2016/anes_timeseries_2016_rawdata.txt',
                      sep='|')

columns = ['V160102', 'V161342', 'V162058x']
labels = ['weight_postfull', 'group', 'party']

anes2016_df = anes2016_df[columns]
anes2016_df.rename(columns=dict(zip(columns, labels)), inplace=True)

anes2016_df = anes2016_df[anes2016_df['party'].isin([10,11,12])]  #Filter out nonvoters

gender2016_df = anes2016_df.groupby(['group', 'party']).agg({'weight_postfull':'sum'})
gender2016_df = gender2016_df.unstack(level=-1)
gender2016_df.columns = gender2016_df.columns.droplevel(0)
gender2016_df = gender2016_df.rename_axis(None, axis=1)

rows = [1,2,3,-9]
races = ['Male', 'Female', 'Other', 'Nonresponse']
gender2016_df.rename(index=dict(zip(rows,races)), inplace=True)
gender2016_df = gender2016_df.loc[races] 

gender2016_df.reset_index(inplace=True)
gender2016_df['total'] = gender2016_df[[10,11,12]].sum(axis=1)
gender2016_df.rename(columns={10:'d',11:'r',12:'o'}, inplace=True)

gender2016_df['dem_frac'] = gender2016_df['d']/gender2016_df['total']
gender2016_df['rep_frac'] = gender2016_df['r']/gender2016_df['total']
gender2016_df['other_frac'] = gender2016_df['o']/gender2016_df['total']

#0 R votes among 'other'
gender2016_df.fillna(0, inplace=True)

#Create electorate fraction
gender2016_df['electorate_frac'] = gender2016_df['total'] / gender2016_df['total'].sum()

gender2016_df['year'] = 2016
gender2016_df['demographic'] = 'gender'
gender2016_df = gender2016_df[['demographic', 'group','year','dem_frac', 'rep_frac',
                               'other_frac','electorate_frac']]

gender2016_df = gender2016_df[gender2016_df['group'].isin(['Male','Female'])]
gender2016_df['group'].replace({'Male':'male','Female':'female'}, inplace=True)

gender2016_df

/Users/psthomas/miniconda2/envs/datascience3/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (790,1129,1131) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,demographic,group,year,dem_frac,rep_frac,other_frac,electorate_frac
0,gender,male,2016,0.454789,0.469016,0.076195,0.462845
1,gender,female,2016,0.520975,0.419133,0.059892,0.524759


In [17]:
baseanes2004_df = pd.read_csv('./data/anes/anes_timeseries_cdf/anes_timeseries_cdf_rawdata.txt',
                      sep='|' )  
#, low_memory=False
#error_bad_lines=False, warn_bad_lines=False

columns = ['VCF0004', 'VCF0009z', 'VCF0104', 'VCF0102', 'VCF0110', 'VCF0705', 'VCF0702']
labels = ['year', 'weight_postfull', 'gender', 'age','education', 'party', 'voted']

baseanes2004_df = baseanes2004_df[columns]
baseanes2004_df.rename(columns=dict(zip(columns, labels)), inplace=True)
baseanes2004_df = baseanes2004_df[baseanes2004_df['year'].isin([2004,2008,2012])]

/Users/psthomas/miniconda2/envs/datascience3/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (13,14,15,18,19,22,23,24,25,26,27,28,29,30,31,32,33,34,35,39,40,41,42,43,44,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,234,255,276,297,298,299,300,301,302,303,304,305,306,307,308,309,310,311,312,313,314,315,316,317,318,319,320,321,322,323,324,325,326,327,328,329,330,331,332,333,334,335,336,337

In [18]:
## Calculate 2004-2012 Democratic Margin By Gender
## source: ANES time series cumulative:
## http://www.electionstudies.org/studypages/anes_timeseries_cdf/anes_timeseries_cdf.htm
gender2004_df = baseanes2004_df[baseanes2004_df['party'].isin([1,2,3])].copy()  #Filter out nonvoters
gender2004_df.rename(columns={'gender':'group'},inplace=True)
gender2004_df = gender2004_df.groupby(['year', 'group', 'party']).agg({'weight_postfull':'sum'})
gender2004_df = gender2004_df.unstack(level=-1)
gender2004_df.columns = gender2004_df.columns.droplevel(0)
gender2004_df = gender2004_df.rename_axis(None, axis=1)

#Reset index, then just relabel by applying to column, no need to select out
gender2004_df.reset_index(inplace=True)
gender2004_df['group'] = gender2004_df['group'].replace({1:'male',2:'female'})
gender2004_df['total'] = gender2004_df[[1,2,3]].sum(axis=1)
gender2004_df.rename(columns={1:'d',2:'r',3:'o'}, inplace=True)

#Divide the number of respondents for each gender by total gender respondents
#to get fraction of electorate
sum_df = gender2004_df.groupby('year')['total'].sum()
def electorate_frac(row):
    row['electorate_frac'] = row['total']/sum_df[row['year']]
    return row 
gender2004_df = gender2004_df.apply(electorate_frac, axis=1)

gender2004_df['dem_frac'] = gender2004_df['d']/gender2004_df['total']
gender2004_df['rep_frac'] = gender2004_df['r']/gender2004_df['total']
gender2004_df['other_frac'] = gender2004_df['o']/gender2004_df['total']

gender2004_df['demographic'] = 'gender'
gender2004_df = gender2004_df[['group','year','dem_frac', 'rep_frac', 
                               'other_frac', 'electorate_frac', 'demographic']]

gender2004_df

,group,year,dem_frac,rep_frac,other_frac,electorate_frac,demographic
0,male,2004,0.478701,0.491888,0.029411,0.519864,gender
1,female,2004,0.444623,0.544912,0.010465,0.480136,gender
2,male,2008,0.505027,0.467885,0.027088,0.429771,gender
3,female,2008,0.561853,0.423652,0.014495,0.570229,gender
4,male,2012,0.486533,0.469894,0.043573,0.478998,gender
5,female,2012,0.549398,0.430607,0.019995,0.521002,gender


In [19]:
#Concat 2016 data on, and then merge with num_nation and turnout data:
gender_df = pd.concat([gender2004_df, gender2016_df])

#Add in number in electorate
electorate_df = national_df[['year','num_nation']]
gender_df = gender_df.merge(electorate_df, on='year', how='left')

#2004-2016 Turnout by Gender
#It's possible to calculate this from ANES data, but it is really off.  See cell below
#Source Current Population Survey data:
#https://www.census.gov/data/tables/2004/demo/voting-and-registration/p20-562-rv.html
#https://www.census.gov/data/tables/2008/demo/voting-and-registration/p20-562-rv.html
#https://www.census.gov/data/tables/2012/demo/voting-and-registration/p20-568.html
#https://www.census.gov/data/tables/2016/demo/voting-and-registration/p20-562-rv.html
#All are percent of citizens voting
gender_turnout = StringIO("""
year,group,turnout
2004,male,62.1
2004,female,65.4
2008,male,61.5
2008,female,65.7
2012,male,59.7
2012,female,63.7
2016,male,59.3
2016,female,63.3
""")
gender_turnout_df = pd.read_csv(gender_turnout, sep=',')
gender_turnout_df = gender_turnout_df.apply(pd.to_numeric, errors='ignore', axis=0)
gender_turnout_df['turnout'] = gender_turnout_df['turnout']/100
gender_df = gender_df.merge(gender_turnout_df, on=['year','group'])

gender_df['dem_margin'] = gender_df['dem_frac'] - gender_df['rep_frac']

order = ['demographic','group', 'year','dem_frac','rep_frac','electorate_frac', 'num_nation', 'dem_margin', 'turnout']

gender_df = gender_df[order]

gender_df

,demographic,group,year,dem_frac,rep_frac,electorate_frac,num_nation,dem_margin,turnout
0,gender,male,2004,0.478701,0.491888,0.519864,122217180,-0.013188,0.621
1,gender,female,2004,0.444623,0.544912,0.480136,122217180,-0.100289,0.654
2,gender,male,2008,0.505027,0.467885,0.429771,131049863,0.037142,0.615
3,gender,female,2008,0.561853,0.423652,0.570229,131049863,0.138201,0.657
4,gender,male,2012,0.486533,0.469894,0.478998,129254383,0.016639,0.597
5,gender,female,2012,0.549398,0.430607,0.521002,129254383,0.118791,0.637
6,gender,male,2016,0.454789,0.469016,0.462845,136670626,-0.014226,0.593
7,gender,female,2016,0.520975,0.419133,0.524759,136670626,0.101842,0.633


In [20]:
turnout_df = baseanes2004_df.copy()
turnout_df = turnout_df.groupby(['year', 'voted']).agg({'weight_postfull': 'sum'})
turnout_df
#This would put 2008 turnout at 70%
#While this only has it at 62%
#http://www.electproject.org/2008g
#            VEP (all ballots) VEP   VAP
#2004 United States	60.7%	60.1%	55.4%
#2008 United States	62.2%	61.6%	56.9%
#2012 United States	58.6%	58.0%	53.6%
#2016 United States	60.2%	59.3%	54.7%

# 2004 812/(812+248+150) = 67.17%
# 2008 1639/(473+208+1639) = 70.6%
# 2012 4282/(4282+1218+413) = 72.4%
# So it does seem that raw surveys overestimate voter turnout substantially.
# So stick with the adjusted CPS Turnout rates from the elections project,
# and the demographic data from them as well when possible. 

weight_postfull
year voted                 
2004 0             150.9553
     1             248.7554
     2             812.2912
2008 0             208.9927
     1             473.0947
     2            1639.9759
2012 0             413.1968
     1            1218.2133
     2            4282.5888

# Age data from CPS, ANES, 2004-2016

I obtain turnout and share of the electorate data frome CPS, the democratic margin by age from the ANES data,
and the number of national voters from the cumulative county data.  


In [33]:
## Calculate 2016 Democratic Margin By Age Group
## source: ANES time series cumulative:
## http://www.electionstudies.org/studypages/anes_timeseries_cdf/anes_timeseries_cdf.htm

# V161267x
# Label: Item name:
# Question:
# PRE: SUMMARY - Respondent age group
# Not applicable; administrative or derived variable
# Not applicable
# Unweighted Frequencies
# ￼01. Age group 18-20 
# 02. Age group 21-24 
# 03. Age group 25-29 
# 04. Age group 30-34 
# 05. Age group 35-39 
# 06. Age group 40-44 
# 07. Age group 45-49 
# 08. Age group 50-54 
# 09. Age group 55-59 
# 10. Age group 60-64 
# 11. Age group 65-69 
# 12. Age group 70-74
# 13. Age group 75 or older
# -1. Inapplicable

anes2016_df = pd.read_csv('./data/anes/anes_timeseries_2016/anes_timeseries_2016_rawdata.txt',
                      sep='|')

columns = ['V160102', 'V161267x', 'V162058x']
labels = ['weight_postfull', 'group', 'party']

anes2016_df = anes2016_df[columns]
anes2016_df.rename(columns=dict(zip(columns, labels)), inplace=True)

anes2016_df = anes2016_df[anes2016_df['party'].isin([10,11,12])]  #Filter out nonvoters

age2016_df = anes2016_df.groupby(['group', 'party']).agg({'weight_postfull':'sum'})
age2016_df = age2016_df.unstack(level=-1)
age2016_df.columns = age2016_df.columns.droplevel(0)
age2016_df = age2016_df.rename_axis(None, axis=1)

# pairs = [[1,2],[3,4],[5,6],[7,8],[9,10],[11,12]]
# labels = {1: "17 - 24", 2: "25 - 34", 3: "35 - 44", 4: "45 - 54", 5: "55 - 64", 6: "65 - 74", 7: "75 +"}

pairs = [[1,2,3],[4,5,6],[7,8,9],[10,11,12]]
labels = {1: "17 - 29", 2: "30 - 44", 3: "45 - 59", 4: "60 +"}

for i, pair in enumerate(pairs):
    age2016_df.loc[labels[i+1]] = age2016_df.loc[pair].sum()
    
age2016_df = age2016_df.loc[labels.values()]


age2016_df.reset_index(inplace=True)
age2016_df['total'] = age2016_df[[10,11,12]].sum(axis=1)
age2016_df.rename(columns={10:'d',11:'r',12:'o'}, inplace=True)

age2016_df['dem_frac'] = age2016_df['d']/age2016_df['total']
age2016_df['rep_frac'] = age2016_df['r']/age2016_df['total']
age2016_df['other_frac'] = age2016_df['o']/age2016_df['total']

#0 R votes among 'other'
age2016_df.fillna(0, inplace=True)

#Create electorate fraction
age2016_df['electorate_frac'] = age2016_df['total'] / age2016_df['total'].sum()


age2016_df['year'] = 2016
age2016_df['demographic'] = 'age'


age2016_df = age2016_df[['demographic', 'group','year','dem_frac', 'rep_frac',
                               'other_frac','electorate_frac']]


age2016_df

/Users/psthomas/miniconda2/envs/datascience3/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (790,1129,1131) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,demographic,group,year,dem_frac,rep_frac,other_frac,electorate_frac
0,age,17 - 29,2016,0.548326,0.351484,0.100191,0.173320
1,age,30 - 44,2016,0.555207,0.345984,0.098809,0.245604
2,age,45 - 59,2016,0.446847,0.479965,0.073187,0.338233
3,age,60 +,2016,0.475273,0.494703,0.030024,0.242843


In [37]:
## Calculate 2004-2012 Democratic Margin By Gender
## source: ANES time series cumulative:
## http://www.electionstudies.org/studypages/anes_timeseries_cdf/anes_timeseries_cdf.htm
# VCF0102

# DEMOGRAPHICS: Respondent - Age Group

# QUESTION:
# ---------
# 1964-1976:  What is your date of birth?
# 1978-1982:  What is the month and year of your birth?
# 1984-LATER: What is the month, day and year of your birth?

# VALID_CODES:
# ------------
# 1.  17 - 24
# 2.  25 - 34
# 3.  35 - 44
# 4.  45 - 54
# 5.  55 - 64
# 6.  65 - 74
# 7.  75 - 99 and over (except 1954)
# MISSING_CODES:
# --------------
# 0.  NA; DK; RF; no Pre IW
# INAP. Inap. question not used

age2004_df = baseanes2004_df[baseanes2004_df['party'].isin([1,2,3])].copy()  #Filter out nonvoters
age2004_df.rename(columns={'age':'group'},inplace=True)
age2004_df = age2004_df.groupby(['year', 'group', 'party']).agg({'weight_postfull':'sum'})
age2004_df = age2004_df.unstack(level=-1)
age2004_df.columns = age2004_df.columns.droplevel(0)
age2004_df = age2004_df.rename_axis(None, axis=1)

age2004_df.fillna(0, inplace=True)
age2004_df.reset_index(inplace=True)

#Interpolate to get correct age ranges
labels = {1: "17 - 29", 2: "30 - 44", 3: "45 - 59", 4: "60 +"}
age2004_df.loc["17 - 29"] = age2004_df.loc[1] + 0.5*age2004_df.loc[2]
age2004_df.loc["30 - 44"] = 0.5*age2004_df.loc[2] + age2004_df.loc[3]
age2004_df.loc["45 - 59"] = age2004_df.loc[4] + 0.5*age2004_df.loc[5]
age2004_df.loc["60 +"] = 0.5*age2004_df.loc[5] + age2004_df.loc[6] + age2004_df.loc[7]

age2004_df['group'] = age2004_df['group'].replace(labels)
age2004_df = age2004_df[age2004_df['group'].isin(labels.values())]
age2004_df['total'] = age2004_df[[1,2,3]].sum(axis=1)
age2004_df.rename(columns={1:'d',2:'r',3:'o'}, inplace=True)

#Divide the number of respondents for each gender by total gender respondents
#to get fraction of electorate
sum_df = age2004_df.groupby('year')['total'].sum()
def electorate_frac(row):
    row['electorate_frac'] = row['total']/sum_df[row['year']]
    return row 
age2004_df = age2004_df.apply(electorate_frac, axis=1)

age2004_df['dem_frac'] = age2004_df['d']/age2004_df['total']
age2004_df['rep_frac'] = age2004_df['r']/age2004_df['total']
age2004_df['other_frac'] = age2004_df['o']/age2004_df['total']

age2004_df['demographic'] = 'age'
age2004_df = age2004_df[['demographic','group','year','dem_frac', 'rep_frac', 
                               'other_frac', 'electorate_frac']]

age2004_df['year'] = age2004_df['year'].astype(int)

age2004_df

,demographic,group,year,dem_frac,rep_frac,other_frac,electorate_frac
0,age,17 - 29,2004,0.429017,0.412320,0.158662,0.123634
1,age,30 - 44,2004,0.581622,0.373218,0.045159,0.255757
2,age,45 - 59,2004,0.420608,0.579392,0.000000,0.322568
3,age,60 +,2004,0.445143,0.554857,0.000000,0.298042
8,age,17 - 29,2008,0.671394,0.325719,0.002887,0.141188
9,age,30 - 44,2008,0.629155,0.350683,0.020162,0.268001
10,age,45 - 59,2008,0.562251,0.423448,0.014301,0.250993
11,age,60 +,2008,0.504923,0.465264,0.029813,0.339818
16,age,17 - 29,2012,0.625693,0.338356,0.035951,0.139413
17,age,30 - 44,2012,0.571377,0.368016,0.060607,0.264247


In [39]:
#Turnout by age, CPS
#Source: http://www.electproject.org/home/voter-turnout/demographics
# Corrected for response bias
# age_turnout = StringIO("""
#     2016	2012	2008	2004
# 18-29	43.4	40.9	48.4	45.0
# 30-44	56.9	56.2	60.7	59.5
# 45-59	66.2	66.0	69.5	69.0
# 60+	71.4	71.2	71.0	69.7
# """)
# Uncorrected for response bias
age_turnout = StringIO("""
group	2016	2012	2008	2004
17 - 29	46.1	45.0	51.1	49.0
30 - 44	58.8	59.5	61.8	62.4
45 - 59	65.9	66.9	68.5	70.0
60 +	70.3	71.8	70.8	71.4
""")

ageturnout_df = pd.read_csv(age_turnout, sep='\t')
ageturnout_df = pd.melt(ageturnout_df, id_vars=["group"], 
                  var_name="year", value_name="turnout")
ageturnout_df = ageturnout_df.apply(pd.to_numeric, errors='ignore', axis=0)
ageturnout_df['turnout'] = ageturnout_df['turnout']/100

age_df = pd.concat([age2004_df, age2016_df])
age_df = age_df.merge(ageturnout_df, on=['group','year'])

#Add in number in electorate
electorate_df = national_df[['year','num_nation']]
age_df = age_df.merge(electorate_df, on='year', how='left')

age_df['dem_margin'] = age_df['dem_frac'] - age_df['rep_frac']

order = ['demographic','group', 'year','dem_frac','rep_frac','electorate_frac', 'num_nation', 'dem_margin', 'turnout']
age_df = age_df[order]

age_df

,demographic,group,year,dem_frac,rep_frac,electorate_frac,num_nation,dem_margin,turnout
0,age,17 - 29,2004,0.429017,0.412320,0.123634,122217180,0.016697,0.490
1,age,30 - 44,2004,0.581622,0.373218,0.255757,122217180,0.208404,0.624
2,age,45 - 59,2004,0.420608,0.579392,0.322568,122217180,-0.158784,0.700
3,age,60 +,2004,0.445143,0.554857,0.298042,122217180,-0.109714,0.714
4,age,17 - 29,2008,0.671394,0.325719,0.141188,131049863,0.345674,0.511
5,age,30 - 44,2008,0.629155,0.350683,0.268001,131049863,0.278472,0.618
6,age,45 - 59,2008,0.562251,0.423448,0.250993,131049863,0.138803,0.685
7,age,60 +,2008,0.504923,0.465264,0.339818,131049863,0.039659,0.708
8,age,17 - 29,2012,0.625693,0.338356,0.139413,129254383,0.287337,0.450
9,age,30 - 44,2012,0.571377,0.368016,0.264247,129254383,0.203361,0.595


# Education Data from CPS, ANES 2004-2016

In [24]:
#Turnout by education, CPS
#Source: http://www.electproject.org/home/voter-turnout/demographics
# Corrected for response bias
# Uncorrected for response bias
age_turnout = StringIO("""
group	2016	2012	2008	2004
Less Than High School	34.3	38.0	39.4	39.5
High School Grad	51.6	52.6	54.9	56.4
Some College to College Grad	67.8	68.4	71.5	72.2
Post-Graduate	80.3	81.4	82.7	84.2
""")

edturnout_df = pd.read_csv(age_turnout, sep='\t')
#Just interpolate Some College to Post-Graduate to deal with lack of data
#in ANES survey:
#Either this, or do weighted average later, but it's impossible to do weighted
#average for 2004 to 2012 because you don't know the correct subpopulation.
#Also note that "Less Than High School" and "High School Grad" leaves out those
#that started high school and didn't finish. . . 
#Weight instead based on US overall values:
#https://en.wikipedia.org/wiki/Educational_attainment_in_the_United_States
#Postgrad: 11.77+3.27+1.77 = 16.81%
#Some college: 58.57%
edturnout_df.set_index('group', inplace=True)
#Simple average overestimates:
# edturnout_df.loc['Some College to Post-Graduate'] = (edturnout_df.loc['Some College to College Grad'] + 
#                                                      edturnout_df.loc['Post-Graduate'])/2
#Weighted average:
edturnout_df.loc['Some College to Post-Graduate'] = (edturnout_df.loc['Some College to College Grad']*0.5857 + 
                                                     edturnout_df.loc['Post-Graduate']*0.1681)/(0.5857+0.1681)
edturnout_df.reset_index(inplace=True)
edturnout_df = edturnout_df.loc[[0,1,4]]
edturnout_df = pd.melt(edturnout_df, id_vars=["group"], 
                  var_name="year", value_name="turnout")
edturnout_df = edturnout_df.apply(pd.to_numeric, errors='ignore', axis=0)
edturnout_df['turnout'] = edturnout_df['turnout']/100



edturnout_df

,group,year,turnout
0,Less Than High School,2016,0.343000
1,High School Grad,2016,0.516000
2,Some College to Post-Graduate,2016,0.705875
3,Less Than High School,2012,0.380000
4,High School Grad,2012,0.526000
5,Some College to Post-Graduate,2012,0.712990
6,Less Than High School,2008,0.394000
7,High School Grad,2008,0.549000
8,Some College to Post-Graduate,2008,0.739976
9,Less Than High School,2004,0.395000


In [25]:
## Calculate 2004-2012 Democratic Margin By Education
## source: ANES time series cumulative:
## http://www.electionstudies.org/studypages/anes_timeseries_cdf/anes_timeseries_cdf.htm

# VCF0110
# DEMOGRAPHICS: Respondent - Education 4-category

# VALID_CODES:
# ------------
# 1.  Grade school or less (0-8 grades)
# 2.  High school (12 grades or fewer, incl. non-college
#       training if applicable)
# 3.  Some college (13 grades or more but no degree; 
#       1948 ONLY: college, no identification of degree
#       status)
# 4.  College or advanced degree (no cases 1948)

#https://nces.ed.gov/fastfacts/display.asp?id=40
#The 6-year graduation rate for first-time, full-time undergraduate students 
#who began seeking a bachelor's degree at a 4-year degree-granting institution 
#in fall 2009 was 59 percent.
# so (finish_college + advanced) = starters*0.59
# some_college = 0.41*starters

ed2004_df = baseanes2004_df[baseanes2004_df['party'].isin([1,2,3])].copy()  #Filter out nonvoters
ed2004_df.rename(columns={'education':'group'},inplace=True)
ed2004_df = ed2004_df.groupby(['year', 'group', 'party']).agg({'weight_postfull':'sum'})
ed2004_df = ed2004_df.unstack(level=-1)
ed2004_df.columns = ed2004_df.columns.droplevel(0)
ed2004_df = ed2004_df.rename_axis(None, axis=1)
ed2004_df.fillna(0, inplace=True)
ed2004_df.reset_index(inplace=True)

#Interpolate to get correct age ranges
labels = {1: "Less Than High School", 2: "High School Grad",
          3: "Some College to Post-Graduate",
          4: "Some College to Post-Graduate"}
ed2004_df['group'].replace(labels, inplace=True)
ed2004_df = ed2004_df.groupby(['year','group']).agg('sum')
ed2004_df.reset_index(inplace=True)

ed2004_df['total'] = ed2004_df[[1,2,3]].sum(axis=1)
ed2004_df.rename(columns={1:'d',2:'r',3:'o'}, inplace=True)

#Divide the number of respondents for each group by total gender respondents
#to get fraction of electorate
sum_df = ed2004_df.groupby('year')['total'].sum()
def electorate_frac(row):
    row['electorate_frac'] = row['total']/sum_df[row['year']]
    return row 
ed2004_df = ed2004_df.apply(electorate_frac, axis=1)
ed2004_df = ed2004_df[ed2004_df['group'].isin(labels.values())]


ed2004_df['dem_frac'] = ed2004_df['d']/ed2004_df['total']
ed2004_df['rep_frac'] = ed2004_df['r']/ed2004_df['total']
ed2004_df['other_frac'] = ed2004_df['o']/ed2004_df['total']

ed2004_df['demographic'] = 'education'
ed2004_df = ed2004_df[['demographic','group','year','dem_frac', 'rep_frac', 
                               'other_frac', 'electorate_frac']]

ed2004_df

,demographic,group,year,dem_frac,rep_frac,other_frac,electorate_frac
0,education,High School Grad,2004,0.393690,0.580143,0.026167,0.296605
1,education,Less Than High School,2004,0.652207,0.347793,0.000000,0.047057
2,education,Some College to Post-Graduate,2004,0.479749,0.501125,0.019126,0.656338
4,education,High School Grad,2008,0.582714,0.407605,0.009682,0.340335
5,education,Less Than High School,2008,0.758134,0.241866,0.000000,0.014612
6,education,Some College to Post-Graduate,2008,0.508247,0.466495,0.025258,0.641328
8,education,High School Grad,2012,0.540501,0.439836,0.019663,0.331596
9,education,Less Than High School,2012,0.690355,0.281942,0.027703,0.013457
10,education,Some College to Post-Graduate,2012,0.503933,0.458739,0.037328,0.644741


In [26]:
## Calculate 2016 Democratic Margin By Education
## source: ANES time series cumulative:
## http://www.electionstudies.org/studypages/anes_timeseries_cdf/anes_timeseries_cdf.htm
# V161270
# What is the highest level of school you have completed or the highest degree you have received?

# FTF Web Total
# 13. Bachelor’s degree (for example: BA, AB, BS)
# 10. Some college but no degree 
# 9. High school graduate- high school diploma or equivalent (for example: GED) 
# 14. Master’s degree (for example: MA, MS, MENG, MED, MSW, MBA) 
# 11. Associate degree in college - occupational/vocational
# program 
# 12. Associate degree in college – academic program
# 8. 12th grade no diploma 
# 16. Doctorate degree (for example: PHD, EDD) 
# 15. Professional school degree (for example: MD, DDS, DVM, LLB, JD)
# 7. 11th grade 
# 6. 10th grade 
# 5. 9th grade 
# 95. Other SPECIFY 
# 4. 7th or 8th grade 
# -9. Refused
anes2016_df = pd.read_csv('./data/anes/anes_timeseries_2016/anes_timeseries_2016_rawdata.txt',
                      sep='|')

columns = ['V160102', 'V161270', 'V162058x']
labels = ['weight_postfull', 'group', 'party']
anes2016_df = anes2016_df[columns]
anes2016_df.rename(columns=dict(zip(columns, labels)), inplace=True)
anes2016_df = anes2016_df[anes2016_df['party'].isin([10,11,12])]  #Filter out nonvoters

ed2016_df = anes2016_df.groupby(['group', 'party']).agg({'weight_postfull':'sum'})
ed2016_df = ed2016_df.unstack(level=-1)
ed2016_df.columns = ed2016_df.columns.droplevel(0)
ed2016_df = ed2016_df.rename_axis(None, axis=1)
ed2016_df.fillna(0, inplace=True)


pairs = [[1,2,3,4],[5,6,7,8,9],[10,11,12,13,14,15,16]]
#Note, I'm including any with some high school in the "High School Grad" category
#because they don't fit anywhere else. . . 
labels = {1: "Less Than High School", 2: "High School Grad",
          3: "Some College to Post-Graduate"}
for i, pair in enumerate(pairs):
    ed2016_df.loc[labels[i+1]] = ed2016_df.loc[pair].sum()

ed2016_df.reset_index(inplace=True)
#Rest are included in sums, so you can remove these:
ed2016_df =  ed2016_df[ed2016_df['group'].isin(labels.values())]
ed2016_df['total'] = ed2016_df[[10,11,12]].sum(axis=1)
ed2016_df.rename(columns={10:'d',11:'r',12:'o'}, inplace=True)
ed2016_df['dem_frac'] = ed2016_df['d']/ed2016_df['total']
ed2016_df['rep_frac'] = ed2016_df['r']/ed2016_df['total']
ed2016_df['other_frac'] = ed2016_df['o']/ed2016_df['total']
ed2016_df.fillna(0, inplace=True)

#Create electorate fraction
ed2016_df['electorate_frac'] = ed2016_df['total'] / ed2016_df['total'].sum()
ed2016_df['year'] = 2016
ed2016_df['demographic'] = 'education'


ed2016_df = ed2016_df[['demographic', 'group','year','dem_frac', 'rep_frac',
                               'other_frac','electorate_frac']]


ed2016_df



/Users/psthomas/miniconda2/envs/datascience3/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (790,1129,1131) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,demographic,group,year,dem_frac,rep_frac,other_frac,electorate_frac
19,education,Less Than High School,2016,0.722333,0.277667,0.000000,0.008721
20,education,High School Grad,2016,0.450025,0.486963,0.063012,0.305485
21,education,Some College to Post-Graduate,2016,0.505691,0.422492,0.071817,0.685794


In [27]:
ed_df = pd.concat([ed2004_df, ed2016_df])

ed_df = ed_df.merge(edturnout_df, on=['year', 'group'])

#Add in number in electorate
electorate_df = national_df[['year','num_nation']]
ed_df = ed_df.merge(electorate_df, on='year', how='left')

ed_df['dem_margin'] = ed_df['dem_frac'] - ed_df['rep_frac']
order = ['demographic','group', 'year','dem_frac','rep_frac','electorate_frac', 'num_nation', 'dem_margin', 'turnout']
ed_df = ed_df[order]

ed_df

,demographic,group,year,dem_frac,rep_frac,electorate_frac,num_nation,dem_margin,turnout
0,education,High School Grad,2004,0.393690,0.580143,0.296605,122217180,-0.186452,0.564000
1,education,Less Than High School,2004,0.652207,0.347793,0.047057,122217180,0.304414,0.395000
2,education,Some College to Post-Graduate,2004,0.479749,0.501125,0.656338,122217180,-0.021377,0.748760
3,education,High School Grad,2008,0.582714,0.407605,0.340335,131049863,0.175109,0.549000
4,education,Less Than High School,2008,0.758134,0.241866,0.014612,131049863,0.516269,0.394000
5,education,Some College to Post-Graduate,2008,0.508247,0.466495,0.641328,131049863,0.041751,0.739976
6,education,High School Grad,2012,0.540501,0.439836,0.331596,129254383,0.100665,0.526000
7,education,Less Than High School,2012,0.690355,0.281942,0.013457,129254383,0.408414,0.380000
8,education,Some College to Post-Graduate,2012,0.503933,0.458739,0.644741,129254383,0.045194,0.712990
9,education,Less Than High School,2016,0.722333,0.277667,0.008721,136670626,0.444665,0.343000


# Combined Demographic Data

In [40]:
demographic_df = pd.concat([race_df, gender_df, age_df, ed_df])
#demographic_df['year'] = demographic_df['year'].round()
demographic_df = demographic_df.round(decimals=4)

demographic_df

,demographic,group,year,dem_frac,rep_frac,electorate_frac,num_nation,dem_margin,turnout
0,race,Non-Hispanic White,2012,0.4300,0.5700,0.7370,129254383,-0.1400,0.6410
1,race,Non-Hispanic Black,2012,0.9700,0.0300,0.1340,129254383,0.9400,0.6640
2,race,Hispanic,2012,0.7300,0.2700,0.0840,129254383,0.4600,0.4800
3,race,Other,2012,0.6100,0.3900,0.0450,129254383,0.2200,0.4900
4,race,Non-Hispanic White,2008,0.4400,0.5600,0.7630,131049863,-0.1200,0.6610
5,race,Non-Hispanic Black,2008,1.0000,0.0000,0.1240,131049863,1.0000,0.6520
6,race,Hispanic,2008,0.7300,0.2700,0.0740,131049863,0.4600,0.4990
7,race,Other,2008,0.8200,0.1800,0.0390,131049863,0.6400,0.4950
8,race,Non-Hispanic White,2004,0.4200,0.5800,0.7920,122217180,-0.1600,0.6720
9,race,Non-Hispanic Black,2004,0.8900,0.1100,0.1120,122217180,0.7800,0.6030


In [41]:
demographic_df.to_csv('./US_Demographic_Presidential_Results_04-16.csv', index=False)